In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [ ]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearc, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [ ]:
from scipy import interpolate

In [ ]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
dataset_name = 'census_categorical'

In [ ]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)
df.columns = [str(i) for i in df.columns]

In [ ]:
# learn graph structure (preferred - fast)
est = TreeSearch(df, root_node=df.columns[0])
# FIXME: 'label' column
dag = est.estimate(estimator_type="tan", class_node='1')

In [ ]:
# alternative graph structure 
if False:
    est2 = TreeSearch(df, root_node=df.columns[0])
    dag2 = est2.estimate(estimator_type="chow-liu")

In [ ]:
# alternative graph structure (slow)
if False:
    est = HillClimbSearch(df)

In [ ]:
best_model = est.estimate() # start_dag=dag)

In [ ]:
nx.draw(best_model, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

In [ ]:
edges = best_model.edges()

In [ ]:
edges

In [ ]:
# there are many choices of parametrization, here is one example
model = BayesianModel(best_model.edges())
model.fit(df, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=0.1)

In [ ]:
print(model.get_cpds('2'))

In [ ]:
# set up train-test sample.
# the test sample is used to calibrate the output of the classifier

random_state = 0
X1_train, X1_test, y1_train, y1_test = train_test_split(data, np.ones(data.shape[0]), test_size=0.35,
                                                        random_state=random_state)

In [ ]:
X1_train.shape

In [ ]:
%%script false --no-raise-error

clf = MLPClassifier(random_state=0, max_iter=1000, early_stopping=True)

In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=250,
    reg_lambda=1,
    gamma=0,
    max_depth=9
)

In [ ]:
n_one = len(X1_train)
n_zero = n_one

In [ ]:
np.random.seed(seed = 0)

# sample data from BN
inference = BayesianModelSampling(model)
df_data = inference.forward_sample(size=n_zero, return_type='dataframe')

df_data.columns = [int(c) for c in df_data.columns]

X0_train = df_data[sorted(df_data.columns)].values

In [ ]:
zeros = np.zeros(n_zero)
ones = np.ones(n_one)

yy = np.concatenate([zeros, ones], axis = 0)
XX = np.concatenate([X0_train, X1_train], axis = 0)

In [ ]:
clf = clf.fit(XX, yy)

In [ ]:
# calibrate the probabilities, using the test sample and a new null sample

In [ ]:
np.random.seed(10)
df_data = inference.forward_sample(size=250000, return_type='dataframe')

df_data.columns = [int(c) for c in df_data.columns]

X0_test = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X0_test)[:, 1]
p1 = clf.predict_proba(X1_test)[:, 1]

In [ ]:
nbins = 100
plt.figure(figsize=(12,7))
plt.hist(p0, bins=100, range=(0,1), alpha=0.5, log=True, density=True);
plt.hist(p1, bins=100, range=(0,1), alpha=0.5, log=True, density=True);

In [ ]:
nbins = 100
binning = np.linspace(0, 1, nbins+1)

hist_p0, bin_edges = np.histogram(p0, binning)
hist_p1, bin_edges = np.histogram(p1, binning)

def poisson_uncertainty(n):
    sigman = np.sqrt(n)
    # correct poisson counts of zero.
    sigman[sigman == 0] = 1.
    return sigman

def fraction_and_uncertainty(a, b, sigma_a, sigma_b):
    absum = a+b
    #frac_a = np.divide(a, absum, out=np.zeros_like(a), where=(absum) != 0)
    #frac_b = np.divide(b, absum, out=np.zeros_like(b), where=(absum) != 0)
    frac_a = a / (a + b)
    frac_b = b / (a + b)
    sigma_fa2 = np.power(frac_b * sigma_a, 2) / np.power(a + b, 2)  +  np.power(frac_a * sigma_b, 2) / np.power(a + b, 2)
    return frac_a, np.sqrt(sigma_fa2)

rest_p0 = np.sum(hist_p0) - hist_p0
rest_p1 = np.sum(hist_p1) - hist_p1

sigma_bin0 = poisson_uncertainty(hist_p0)
sigma_rest0 = poisson_uncertainty(rest_p0)

sigma_bin1 = poisson_uncertainty(hist_p1)
sigma_rest1 = poisson_uncertainty(rest_p1)

frac0, sigma_frac0 = fraction_and_uncertainty(hist_p0, rest_p0, sigma_bin0, sigma_rest0)
frac1, sigma_frac1 = fraction_and_uncertainty(hist_p1, rest_p1, sigma_bin1, sigma_rest1)

p1calib, sigma_p1calib = fraction_and_uncertainty(frac1, frac0, sigma_frac1, sigma_frac0)

sample_weight = 1 / (sigma_p1calib * sigma_p1calib)
sample_weight /= min(sample_weight)

#sample_weight

In [ ]:
# we recalibrate per probability bin. NO interpolation (not valid in highest bin)
#hist_p0, bin_edges = np.histogram(p0, bins=nbins, range=(0, 1))
#hist_p1, bin_edges = np.histogram(p2, bins=nbins, range=(0, 1)) #### !!!! p2
bin_centers = bin_edges[:-1] + 0.5/nbins

hnorm_p0 = hist_p0 / sum(hist_p0)
hnorm_p1 = hist_p1 / sum(hist_p1)
hnorm_sum = hnorm_p0 + hnorm_p1
p1cb = np.divide(hnorm_p1, hnorm_sum, out=np.zeros_like(hnorm_p1), where=hnorm_sum != 0)
# self.p1cb = p1cb, bin_centers

# use isotonic regression to smooth out potential fluctuations in the p1 values
# isotonic regression assumes that p1 can only be a rising function.
# I’m assuming that if a classifier predicts a higher probability, the calibrated probability
# will also be higher. This may not always be right, but I think generally it is a safe one.
iso_reg = IsotonicRegression(y_min=0, y_max=1).fit(bin_centers, p1calib, sample_weight)
p1pred = iso_reg.predict(bin_centers)

# calibrated probabilities
p1f_ = interpolate.interp1d(
    bin_edges[:-1], 
    p1pred, 
    kind='previous', 
    bounds_error=False, 
    fill_value="extrapolate"
)

p1pred = p1f_(bin_centers)

In [ ]:
p1lin = p1f_(bin_centers)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(bin_centers, p1cb, label='p1cb')
plt.plot(bin_centers, p1pred, label='p1pred')
plt.plot(bin_centers, bin_centers, label='bin_centers')
plt.plot(bin_centers, p1lin, label='p1lin')
plt.legend();

In [ ]:
maxp1 = p1f_(0.995)

In [ ]:
max_weight = maxp1 / (1. - maxp1)
max_weight

In [ ]:
# validation - part 1: check if reweighting works okay

In [ ]:
np.random.seed(1)

# sample data from BN
inference = BayesianModelSampling(model)

df_data = inference.forward_sample(size=250000, return_type='dataframe')

df_data.columns = [int(c) for c in df_data.columns]

X_test = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X_test)[:, 1]
nominator = p1f_(p0)
denominator = 1 - nominator
weight = np.divide(nominator, denominator, out=np.ones_like(nominator), where=denominator != 0)

In [ ]:
len(X_test), sum(weight)

In [ ]:
%%script false --no-raise-error

keep = weight == max_weight
same = weight != max_weight
ratio = (250000 - np.sum(weight[same])) / np.sum(weight[keep])
np.sum(weight[same]), np.sum(weight[keep])

In [ ]:
plt.hist(weight, bins=100, log=True);

In [ ]:
#data, sample_weights = self._sample_no_transform(n_samples, random_state)
pop = np.asarray(range(X_test.shape[0]))
probs = weight/np.sum(weight)
sample = choices(pop, probs, k=X_test.shape[0])
Xtrans = X_test[sample]

In [ ]:
p0 = clf.predict_proba(Xtrans)[:, 1]
p1 = clf.predict_proba(X1_test)[:, 1]

In [ ]:
plt.figure(figsize=(12,7))
plt.hist(p0, bins=100, range=(0,1), alpha=0.5, density=True); #, weights=weight)#, log=True)
plt.hist(p1, bins=100, range=(0,1), alpha=0.5, density=True);

In [ ]:
# validation - part 2: plot distributions

In [ ]:
i = 1
plt.figure(figsize=(12,7))
plt.hist(X_test[:, i], bins=100, range=(0,1), alpha=0.5, density=True);#, log=True)
plt.hist(X1_test[:, i], bins=100, range=(0,1), alpha=0.5, density=True);


In [ ]:
# validation part 3: check number of duplicates

In [ ]:
np.random.seed(2)
df_data = inference.forward_sample(size=500000, return_type='dataframe')
df_data.columns = [int(c) for c in df_data.columns]
X10k = df_data[sorted(df_data.columns)].values

In [ ]:
p0 = clf.predict_proba(X10k)[:, 1]
nominator = p1f_(p0)
denominator = 1 - nominator
weight = np.divide(nominator, denominator, out=np.ones_like(nominator), where=denominator != 0)

In [ ]:
sum(weight)

In [ ]:
pop = np.asarray(range(X10k.shape[0]))
probs = weight/np.sum(weight)
sample = choices(pop, probs, k=X10k.shape[0])
Xtrans = X10k[sample]

In [ ]:
u, c = np.unique(Xtrans, axis=0, return_counts=True)

In [ ]:
counts = np.sort(c)[::-1] / 50

In [ ]:
counts

In [ ]:
u, c = np.unique(data, axis=0, return_counts=True)

In [ ]:
c2 = np.sort(c)[::-1] 

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(list(range(40)), c2[:40], alpha=0.5)
plt.bar(list(range(40)), counts[:40], alpha=0.5)

# run sdgym

In [ ]:
df = pd.DataFrame(Xtrans)
df.to_csv(f'{dataset_name}_test.csv', index=False)

In [ ]:
def KDECopulaNNPdf_RoundCategorical(real_data, categorical_columns, ordinal_columns, times=None):
    df = pd.read_csv(f'{dataset_name}_test.csv')
    data = df.values[:real_data.shape[0]]
    return data

In [ ]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    IdentitySynthesizer,
    IndependentSynthesizer,
    PrivBNSynthesizer,
    KDECopulaNNPdf_RoundCategorical,
#     CTGANSynthesizer,
]

In [ ]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name])

In [ ]:
scores

In [ ]:
scores.tail(3)